In [1]:
import tensorflow as tf
print(tf.__version__)

1.15.0


<font size="5">**Requirements to run this project**</font>
* <font size="3">Python 2 - 3.7 </font>
* <dd><font size="3">Tensorflow 1.3 - 1.15 </font></dd>
* <dd><font size="3">MySQL</font></dd>
* <dd><font size="3">sklearn, numpy </font></dd>
* <dd><font size="3">Pyro4 </font></dd>

Check Tensorflow version, it should be (1.15) in order to work properly with the slim DNN models



<font size="5">**Importing libraries and python scripts**</font>
<br /><br />
<font size="3">Before starting, carefully go through the Setup in the README.md to obtain necessary images and checkpoints.  
</font>

In [2]:
#!pip install mysql-connector-python
#!pip install --user mysql-python
#!pip install Pyro4
#!pip install --upgrade pip
#!pip install --user tensorflow==1.15
import mysql
import mysql.connector as mysql
from nets import nets_factory
from preprocessing import preprocessing_factory
import model
import analysis
import os
import inference
import image_generator
import numpy as np

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



<font size="3"> If some libraries are not installed, install them using pip </font>

<font size="5">**Starting the image generator class**</font>
<br /><br />
<font size="3">This code starts the image generator class and can be used to check if the <u>images</u> and <u>dataset files</u> are stored properly and are accessible.</font>

In [3]:
image_generator.main()
imgGen = image_generator.ImageGenerator('images/val/images', 'images/val/val.txt', 'images/val/synset_words.txt')
label_list = imgGen.get_label_list()      # this is the correct label list that will be used to evaluate DNN performance

images/val/images\ILSVRC2012_val_00003283.JPEG hoopskirt, crinoline
images/val/images\ILSVRC2012_val_00006076.JPEG trench coat
images/val/images\ILSVRC2012_val_00035348.JPEG magpie
images/val/images\ILSVRC2012_val_00011321.JPEG puffer, pufferfish, blowfish, globefish
images/val/images\ILSVRC2012_val_00042235.JPEG sorrel
images/val/images\ILSVRC2012_val_00007991.JPEG sidewinder, horned rattlesnake, Crotalus cerastes
images/val/images\ILSVRC2012_val_00048761.JPEG English setter
images/val/images\ILSVRC2012_val_00045261.JPEG Border collie
images/val/images\ILSVRC2012_val_00029748.JPEG running shoe
images/val/images\ILSVRC2012_val_00008217.JPEG komondor


<font size="5">**Testing inference on one image**</font>
<br /><br />
<font size="3">This code initializes the models and infers the first image with each of them. It is used to check if <u>model checkpoints</u> are accessible.</font>

In [4]:
model_name = 'inception_v4'
model = inference.DNN_Model('tf', model_name, 'model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt')
results = model.time_model(['images/val/images/ILSVRC2012_val_00000001.JPEG'], 1, [])
print(results)

model_name = 'resnet_v1_152'
model = inference.DNN_Model('tf', model_name, 'model_data/tensorflow/checkpoints/resnet_v1_152/resnet_v1_152.ckpt')
results = model.time_model(['images/val/images/ILSVRC2012_val_00000001.JPEG'], 1, [])
print(results)

model_name = 'mobilenet_v1'
model = inference.DNN_Model('tf', model_name, 'model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt')
results = model.time_model(['images/val/images/ILSVRC2012_val_00000001.JPEG'], 1, [])
print(results)

Skipping first inference...

Preprocessing image: 1 of 1


INFO:tensorflow:Scale of 0 disables regularizer.

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.flatten instead.

INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 1 of 1	Average Time: 858.0
Done!

Preprocessing image: 1 of 1
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 1 of 1	Average Time: 968.0
[([968.0], [[0.9475860595703125, 0.011890850029885769, 0.0072621447034180164, 0.0031914273276925087, 0.003110471647232771, 0.002808748045936227, 0.0014050431782379746, 0.0009503407636657357, 0.0008339508203789592, 0.0006610105047002435, 0.0006372712086886168, 0.0005837938515469432, 0.0005329005070962012, 0.00038708027568645775, 0.0003691568272188306, 0.0003241834929212928, 0.00015922164311

<font size="5">**Conecting to MySQL**</font>
<br /><br />
<font size="3">This code can be used to check if connection to MySQL can be established. You should insert your own <font color = "green">hostname</font>, <font color = "green">username</font>, and <font color = "green">password</font>. These values should also be used in *connect_db* method in *database.py* file (change it).</font>

In [5]:
db = mysql.connect(
    host = "127.0.0.1", #could be "localhost"
    user = "root",
    passwd = "rootpsw"
)

<font size="5">**Obtaining inference results**</font>
<br /><br />
<font size="3">This code retrieves checkpoints for the models and obtains <font color = "green">inference results</font> on selected images.</font>


In [ ]:
model_i_name = 'inception_v4'
model_i = inference.DNN_Model('tf', model_i_name, 'model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt')

model_r_name = 'resnet_v1_152'
model_r = inference.DNN_Model('tf', model_r_name, 'model_data/tensorflow/checkpoints/resnet_v1_152/resnet_v1_152.ckpt')

model_m_name = 'mobilenet_v1'
model_m = inference.DNN_Model('tf', model_m_name, 'model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt')

models = [model_i, model_r, model_m]
model_names = [model_i_name, model_r_name, model_m_name]
results = []

img_nums = []
img_path_list = []
img_labels = []
for i in range(1, 50):                           # here, the range of images can be selected
    img_nums.append(i)
for i in range(len(img_nums)):
    img_path, img_label = imgGen.get_image_data(img_nums[i])
    img_path_list.append(img_path)
    img_labels.append(img_label)
for i in range(len(models)):
    results.append(models[i].time_model(img_path_list, 1, []))                # 0 - inception, 1 - resnet, 2 - mobilenet

Skipping first inference...
Preprocessing image: 1 of 1
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 1 of 1	Average Time: 943.0
Done!

Preprocessing image: 1 of 49
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 1 of 49	Average Time: 721.0
Preprocessing image: 2 of 49
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 2 of 49	Average Time: 764.0
Preprocessing image: 3 of 49
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 3 of 49	Average Time: 829.0
Preprocessing image: 4 of 49
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tens

Image: 36 of 49	Average Time: 728.0
Preprocessing image: 37 of 49
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 37 of 49	Average Time: 721.0
Preprocessing image: 38 of 49
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 38 of 49	Average Time: 728.0
Preprocessing image: 39 of 49
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 39 of 49	Average Time: 904.0
Preprocessing image: 40 of 49
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 40 of 49	Average Time: 789.0
Preprocessing image: 41 of 49
INFO:tensorflow:Scale of 0 disables regularize

<font size="3">In these cells, results are processed to obtain <font color = "green">top-1</font> and <font color = "green">top-5 satisfaction (0 or 1)</font> for each image by each of the DNNs, as well as <font color = "green">inference times</font>.

In [ ]:
top_1 = [[], [], []]
top_5 = [[], [], []]
performance = [[], [], []]

In [8]:
for j in range(len(models)):
    for i in range(len(img_nums)):
        performance[j].append(results[j][i][0][0])
        top_1[j].append(int(analysis.top_n_correct(np.array(results[j][i][1][0]), np.array(results[j][i][2][0]), img_labels[i], label_list, 1)))
        top_5[j].append(int(analysis.top_n_correct(np.array(results[j][i][1][0]), np.array(results[j][i][2][0]), img_labels[i], label_list, 5)))

<font size="5">**Creating a database**</font>
<br /><br />
<font size="3">Here, we create the database that will hold <font color = "green">image names</font> in the *images* table and <font color = "green">inference results</font> in the *exec_data* table.</font>


In [9]:
#Change host, user, passwd for your own

db = mysql.connect(
    host = "127.0.0.1",
    user = "root",
    passwd = "Vj55577022077555."
)

cursor = db.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS inference")

db = mysql.connect(
    host = "127.0.0.1",
    user = "root",
    passwd = "Vj55577022077555.",
    database = "inference"
)

cursor = db.cursor()

<font size="3">Inserting the image data</font>

In [10]:
cursor.execute("CREATE TABLE IF NOT EXISTS images (img_num int(5) NOT NULL, filename varchar(35) NOT NULL, KEY img_num (img_num)) ENGINE=InnoDB;")

filename = []

for i in range(len(img_nums)):
    filename.append(imgGen.get_image_filename(img_nums[i]))
for i in range(len(img_nums)):
    query = "INSERT INTO images (img_num, filename) VALUES (%s, %s)"
    values = (img_nums[i], filename[i])
    cursor.execute(query, values)

db.commit()

<font size="3">Inserting the inference results</font>

In [11]:
cursor.execute("CREATE TABLE IF NOT EXISTS exec_data (img_num int(5) NOT NULL, model_name varchar(14) NOT NULL, top_1 int(1) NOT NULL, top_5 int(1) NOT NULL, performance int(7) NOT NULL, KEY img_num (img_num)) ENGINE=InnoDB;")

for i in range(len(img_nums)):
    for j in range(len(models)):
        query = "INSERT INTO exec_data (img_num, model_name, top_1, top_5, performance) VALUES (%s, %s, %s, %s, %s)"
        values = (img_nums[i], model_names[j], top_1[j][i], top_5[j][i], performance[j][i])
        cursor.execute(query, values)

db.commit()

<font size="5">**Obtaining information from the database**</font>
<br /><br />
<font size="3">We use *database.py* to obtain the <font color="green">best top-1</font> and <font color="green">top-5</font> models for each image.</font>

In [12]:
import database as db

In [13]:
myDB = db.connect_db("inference")
list_model = ["mobilenet_v1", "inception_v4", "resnet_v1_152"]

for i in range(len(img_nums)):
    print('Best top-1 model for img num \''+str(img_nums[i])+'\' is ')
    print(db.determine_best_top_n_model("inference", img_nums[i], list_model, 1))
    print('\n' * 1)
    print('Best top-5 model for img num =\''+str(img_nums[i])+'\' is ')
    print(db.determine_best_top_n_model("inference", img_nums[i], list_model, 5))
    print('\n' * 2)

Best top-1 model for img num '1' is 
mobilenet_v1


Best top-5 model for img num ='1' is 
mobilenet_v1



Best top-1 model for img num '2' is 
failed


Best top-5 model for img num ='2' is 
mobilenet_v1



Best top-1 model for img num '3' is 
mobilenet_v1


Best top-5 model for img num ='3' is 
mobilenet_v1



Best top-1 model for img num '4' is 
mobilenet_v1


Best top-5 model for img num ='4' is 
mobilenet_v1



Best top-1 model for img num '5' is 
inception_v4


Best top-5 model for img num ='5' is 
mobilenet_v1



Best top-1 model for img num '6' is 
inception_v4


Best top-5 model for img num ='6' is 
inception_v4



Best top-1 model for img num '7' is 
mobilenet_v1


Best top-5 model for img num ='7' is 
mobilenet_v1



Best top-1 model for img num '8' is 
failed


Best top-5 model for img num ='8' is 
inception_v4



Best top-1 model for img num '9' is 
mobilenet_v1


Best top-5 model for img num ='9' is 
mobilenet_v1



Best top-1 model for img num '10' is 
mobilenet_v1


Best 

<font size="3">This function prints out the contents of the *images* table of the database.</font>

In [14]:
for i in range(len(filename)):
    print(db.get_img_num_database("inference", filename[i]))

(1, 'ILSVRC2012_val_00000001.JPEG')
(2, 'ILSVRC2012_val_00000002.JPEG')
(3, 'ILSVRC2012_val_00000003.JPEG')
(4, 'ILSVRC2012_val_00000004.JPEG')
(5, 'ILSVRC2012_val_00000005.JPEG')
(6, 'ILSVRC2012_val_00000006.JPEG')
(7, 'ILSVRC2012_val_00000007.JPEG')
(8, 'ILSVRC2012_val_00000008.JPEG')
(9, 'ILSVRC2012_val_00000009.JPEG')
(10, 'ILSVRC2012_val_00000010.JPEG')
